In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import json
import datetime

## Movie dataset

In [22]:
movie_ori = pd.read_csv(r'movies_initial.csv')

In [23]:
movie_ori

,imdbID,title,year,rating,runtime,genre,released,director,writer,cast,...,imdbRating,imdbVotes,poster,plot,fullplot,language,country,awards,lastupdated,type
0,1,Carmencita,1894,NOT RATED,1 min,"Documentary, Short",NaN,William K.L. Dickson,NaN,Carmencita,...,5.9,1032.0,https://m.media-amazon.com/images/M/MV5BMjAzND...,Performing on what looks like a small wooden s...,Performing on what looks like a small wooden s...,NaN,USA,NaN,2015-08-26 00:03:45.040000000,movie
1,5,Blacksmith Scene,1893,UNRATED,1 min,Short,1893-05-09,William K.L. Dickson,NaN,"Charles Kayser, John Ott",...,6.2,1189.0,NaN,Three men hammer on an anvil and pass a bottle...,A stationary camera looks at a large anvil wit...,NaN,USA,1 win.,2015-08-26 00:03:50.133000000,movie
2,3,Pauvre Pierrot,1892,NaN,4 min,"Animation, Comedy, Short",1892-10-28,�mile Reynaud,NaN,NaN,...,6.7,566.0,NaN,"One night, Arlequin come to see his lover Colo...","One night, Arlequin come to see his lover Colo...",NaN,France,NaN,2015-08-12 00:06:02.720000000,movie
3,8,Edison Kinetoscopic Record of a Sneeze,1894,NaN,1 min,"Documentary, Short",1894-01-09,William K.L. Dickson,NaN,Fred Ott,...,5.9,988.0,NaN,A man (Thomas Edison's assistant) takes a pinc...,A man (Edison's assistant) takes a pinch of sn...,NaN,USA,NaN,2015-08-10 00:21:07.127000000,movie
4,10,Employees Leaving the Lumi�re Factory,1895,NaN,1 min,"Documentary, Short",1895-03-22,Louis Lumi�re,NaN,NaN,...,6.9,3469.0,NaN,A man opens the big gates to the Lumi�re facto...,A man opens the big gates to the Lumi�re facto...,NaN,France,NaN,2015-08-26 00:03:56.603000000,movie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46009,5005684,The New Classmate,2015,NaN,NaN,"Drama, Family",2015-10-11,Ashwiny Iyer Tiwari,NaN,"Swara Bhaskar, Ratna Pathak, Pankaj Tripathy",...,NaN,NaN,NaN,NaN,NaN,Hindi,India,NaN,2015-09-10 22:28:58.883000000,movie
46010,5013056,Untitled Christopher Nolan Project,2017,NaN,NaN,Sci-Fi,2017-07-21,Christopher Nolan,"Christopher Nolan (screenplay), Christopher Nolan",NaN,...,NaN,NaN,NaN,Plot is being kept under wraps.,Plot is being kept under wraps.,English,"USA, UK",NaN,2015-09-11 17:15:04.527000000,movie
46011,5013980,Die Turnschuh-Giganten,2016,NaN,NaN,"Biography, Drama, War",NaN,Oliver Dommenget,Christian Schnalke,"Ken Duken, Florian Bartholom�i, Torben Liebrec...",...,NaN,NaN,NaN,NaN,NaN,German,Germany,NaN,2015-09-12 23:13:48.997000000,movie
46012,5016760,The Monster of Mangatiti,2015,NaN,NaN,NaN,2015-09-06,NaN,John Banas,"Greta Gregory, Mark Mitchinson, Flynn Allan, H...",...,NaN,NaN,NaN,Heather Walsh tells her story of the time she ...,Heather Walsh tells her story of the time she ...,English,New Zealand,NaN,2015-09-12 03:47:27.780000000,movie


In [31]:
movie_validrating = movie_ori.dropna(subset=['imdbRating'])
movie_validrating

,imdbID,title,year,rating,runtime,genre,released,director,writer,cast,...,imdbRating,imdbVotes,poster,plot,fullplot,language,country,awards,lastupdated,type
0,1,Carmencita,1894,NOT RATED,1 min,"Documentary, Short",NaN,William K.L. Dickson,NaN,Carmencita,...,5.9,1032.0,https://m.media-amazon.com/images/M/MV5BMjAzND...,Performing on what looks like a small wooden s...,Performing on what looks like a small wooden s...,NaN,USA,NaN,2015-08-26 00:03:45.040000000,movie
1,5,Blacksmith Scene,1893,UNRATED,1 min,Short,1893-05-09,William K.L. Dickson,NaN,"Charles Kayser, John Ott",...,6.2,1189.0,NaN,Three men hammer on an anvil and pass a bottle...,A stationary camera looks at a large anvil wit...,NaN,USA,1 win.,2015-08-26 00:03:50.133000000,movie
2,3,Pauvre Pierrot,1892,NaN,4 min,"Animation, Comedy, Short",1892-10-28,�mile Reynaud,NaN,NaN,...,6.7,566.0,NaN,"One night, Arlequin come to see his lover Colo...","One night, Arlequin come to see his lover Colo...",NaN,France,NaN,2015-08-12 00:06:02.720000000,movie
3,8,Edison Kinetoscopic Record of a Sneeze,1894,NaN,1 min,"Documentary, Short",1894-01-09,William K.L. Dickson,NaN,Fred Ott,...,5.9,988.0,NaN,A man (Thomas Edison's assistant) takes a pinc...,A man (Edison's assistant) takes a pinch of sn...,NaN,USA,NaN,2015-08-10 00:21:07.127000000,movie
4,10,Employees Leaving the Lumi�re Factory,1895,NaN,1 min,"Documentary, Short",1895-03-22,Louis Lumi�re,NaN,NaN,...,6.9,3469.0,NaN,A man opens the big gates to the Lumi�re facto...,A man opens the big gates to the Lumi�re facto...,NaN,France,NaN,2015-08-26 00:03:56.603000000,movie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45954,4938416,LEGO DC Super Heroes: Justice League - Attack ...,2015,NaN,77 min,NaN,2015-08-25,Rick Morales,"Neal Adams (character created by: Man-Bat), Jo...","Dee Bradley Baker, Troy Baker, John DiMaggio, ...",...,6.1,119.0,https://m.media-amazon.com/images/M/MV5BNjU5OD...,"Get ready for the bricks to fly when Superman,...","Get ready for the bricks to fly when Superman,...",NaN,USA,NaN,2015-09-13 00:05:19.837000000,movie
45974,4957236,Cosmos Laundromat,2015,NaN,12 min,"Animation, Short",2015-08-10,Mathieu Auvray,Esther Wouda,"Pierre Bokma, Reinout Scholten van Aschat",...,8.8,10.0,https://m.media-amazon.com/images/M/MV5BODUwMj...,"On a desolate island, suicidal sheep Franck me...","On a desolate island, suicidal sheep Franck me...",English,Netherlands,NaN,2015-09-03 02:18:10.507000000,movie
45982,4967094,Our Times,2015,NaN,NaN,Romance,2015-08-13,Frankie Chen,NaN,"Joe Chen, Dewi Chien, Vivian Sung, Darren Wang",...,7.4,7.0,NaN,Love grows where it isn't expected in this end...,The movie takes us on a 20 year journey throug...,Mandarin,Taiwan,NaN,2015-08-27 22:32:56.313000000,movie
45998,4987556,Thani Oruvan,2015,NaN,160 min,"Action, Thriller",2015-08-28,M. Raja,"A.N. Balakrishnan, M. Raja, M. Raja, D. Suresh...","Abhinaya, Junior Balaiya, Mugdha Godse, V. Jay...",...,9.4,25.0,NaN,Revolving around a police officer who is passi...,Thoroughly frustrated with Industrialists and ...,Tamil,India,NaN,2015-09-02 23:40:25.913000000,movie


In [32]:
movie_validrating.columns

Index(['imdbID', 'title', 'year', 'rating', 'runtime', 'genre', 'released',
       'director', 'writer', 'cast', 'metacritic', 'imdbRating', 'imdbVotes',
       'poster', 'plot', 'fullplot', 'language', 'country', 'awards',
       'lastupdated', 'type'],
      dtype='object')

In [35]:
movie = movie_validrating.drop(['plot','fullplot','lastupdated'], axis=1)
movie

,imdbID,title,year,rating,runtime,genre,released,director,writer,cast,metacritic,imdbRating,imdbVotes,poster,language,country,awards,type
0,1,Carmencita,1894,NOT RATED,1 min,"Documentary, Short",NaN,William K.L. Dickson,NaN,Carmencita,NaN,5.9,1032.0,https://m.media-amazon.com/images/M/MV5BMjAzND...,NaN,USA,NaN,movie
1,5,Blacksmith Scene,1893,UNRATED,1 min,Short,1893-05-09,William K.L. Dickson,NaN,"Charles Kayser, John Ott",NaN,6.2,1189.0,NaN,NaN,USA,1 win.,movie
2,3,Pauvre Pierrot,1892,NaN,4 min,"Animation, Comedy, Short",1892-10-28,�mile Reynaud,NaN,NaN,NaN,6.7,566.0,NaN,NaN,France,NaN,movie
3,8,Edison Kinetoscopic Record of a Sneeze,1894,NaN,1 min,"Documentary, Short",1894-01-09,William K.L. Dickson,NaN,Fred Ott,NaN,5.9,988.0,NaN,NaN,USA,NaN,movie
4,10,Employees Leaving the Lumi�re Factory,1895,NaN,1 min,"Documentary, Short",1895-03-22,Louis Lumi�re,NaN,NaN,NaN,6.9,3469.0,NaN,NaN,France,NaN,movie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45954,4938416,LEGO DC Super Heroes: Justice League - Attack ...,2015,NaN,77 min,NaN,2015-08-25,Rick Morales,"Neal Adams (character created by: Man-Bat), Jo...","Dee Bradley Baker, Troy Baker, John DiMaggio, ...",NaN,6.1,119.0,https://m.media-amazon.com/images/M/MV5BNjU5OD...,NaN,USA,NaN,movie
45974,4957236,Cosmos Laundromat,2015,NaN,12 min,"Animation, Short",2015-08-10,Mathieu Auvray,Esther Wouda,"Pierre Bokma, Reinout Scholten van Aschat",NaN,8.8,10.0,https://m.media-amazon.com/images/M/MV5BODUwMj...,English,Netherlands,NaN,movie
45982,4967094,Our Times,2015,NaN,NaN,Romance,2015-08-13,Frankie Chen,NaN,"Joe Chen, Dewi Chien, Vivian Sung, Darren Wang",NaN,7.4,7.0,NaN,Mandarin,Taiwan,NaN,movie
45998,4987556,Thani Oruvan,2015,NaN,160 min,"Action, Thriller",2015-08-28,M. Raja,"A.N. Balakrishnan, M. Raja, M. Raja, D. Suresh...","Abhinaya, Junior Balaiya, Mugdha Godse, V. Jay...",NaN,9.4,25.0,NaN,Tamil,India,NaN,movie


## Web Scraping

In [2]:
# Use web scraping to obtain the ranking(#/500) and the movie name
def moviescraping(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    movie_rankname = soup.select('h3.lister-item-header')
    for i in range(0, len(movie_rankname)):
        string_list = movie_rankname[i].get_text().split("\n")
        movie_title = string_list[2]
        ranking = int(string_list[1].replace('.',''))
        top500dict[ranking] = movie_title

## Caching

In [3]:
CACHE_FILENAME = "cache.json"

def open_cache():
    ''' opens the cache file if it exists and loads the JSON into
    a dictionary, which it then returns.
    if the cache file doesn't exist, creates a new cache dictionary
    Parameters
    ----------
    None
    Returns
    -------
    The opened cache
    '''
    try:
        cache_file = open(CACHE_FILENAME, 'r')
        cache_contents = cache_file.read()
        cache_dict = json.loads(cache_contents)
        cache_file.close()
    except:
        cache_dict = {}
    return cache_dict

def save_cache(cache_dict):
    ''' saves the current state of the cache to disk
    Parameters
    ----------
    cache_dict: dict
        The dictionary to save
    Returns
    -------
    None
    '''
    dumped_json_cache = json.dumps(cache_dict)
    fw = open(CACHE_FILENAME,"w")
    fw.write(dumped_json_cache)
    fw.close()

In [37]:
def load_webscraping_data():
    if open_cache():  # get data from cache
        return open_cache()
       
    else: # first time from web scraping
        moviescraping('https://www.imdb.com/list/ls050782187/')
        moviescraping('https://www.imdb.com/list/ls050782187/?sort=list_order,asc&st_dt=&mode=detail&page=2')
        moviescraping('https://www.imdb.com/list/ls050782187/?sort=list_order,asc&st_dt=&mode=detail&page=3')
        moviescraping('https://www.imdb.com/list/ls050782187/?sort=list_order,asc&st_dt=&mode=detail&page=4')
        moviescraping('https://www.imdb.com/list/ls050782187/?sort=list_order,asc&st_dt=&mode=detail&page=5')
        save_cache(top500dict)
        return top500dict

In [38]:
top500dict = {}
CACHE = open_cache()

## first time load from web scraping, run without the cache.json file
t1 = datetime.datetime.now().timestamp()
load_webscraping_data()
t2 = datetime.datetime.now().timestamp()

## second time load from cache
t3 = datetime.datetime.now().timestamp()
load_webscraping_data()
t4 = datetime.datetime.now().timestamp()


print("time without caching: ", (t2 - t1) * 1000, "ms")
print("time with caching: ", (t4 - t3) * 1000, "ms")

time without caching:  19113.265991210938 ms
time with caching:  0.4279613494873047 ms
